In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.3/spark-3.2.3-bin-hadoop3.2.tgz
!tar xf spark-3.2.3-bin-hadoop3.2.tgz
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.3-bin-hadoop3.2"
!pip install -q findspark
!pip install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 17.7 MB/s eta 0:00:00


### Almacenamiento en caché

In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [3]:
rdd = sc.parallelize([item for item in range(10)])

In [7]:
# Para definir el nivel de persistencia. El nivel depende del caso, para optimizar aplicaciones
from pyspark.storagelevel import StorageLevel

In [6]:
# Para persistir RDD sólo en memoria
# Memory only = sólo caché
rdd.persist(StorageLevel.MEMORY_ONLY)

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [8]:
# Para cambiar nivel de persistencia, primero hay que hacer unpersist
rdd.unpersist()
rdd.persist(StorageLevel.DISK_ONLY)

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [ ]:
# rdd.cache() = rdd.persist(memory_only)
rdd.unpersist()
rdd.cache()

### Particionado de datos

Si la cantidad de particiones es pequeña, usaremos solo unas pocas CPUs/núcleos en una gran cantidad de datos

Si la cantidad de particiones es demasiado grande, se utilizará más recursos de los que realmente se necesitan

In [ ]:
# HashPartitioner: selecciona un hash para cada clave de los elementos
# RangePartitioner: divide el RDD en rangos iguales

In [9]:
rdd = sc.parallelize(['x','y','z'])

In [10]:
# ejemplo
hola = 'Hola'
hash(hola)

-2584198304634502723

In [11]:
# índice = hash(item) % num_particiones

In [12]:
num_particiones = 6

In [13]:
hash('x') % num_particiones

0

In [14]:
hash('y') % num_particiones

4

In [15]:
hash('z') % num_particiones

1

### Mezcla de datos (shuffling)

Se pueden crear nuevas particiones o fusionar. Todo el movimiento de datos para el reparticionamiento se denomina shiffling. Es costoso, tenerlo en cuanta al crear jobs.

### Broadcast variables

Las variables broadcast son variables compartidas entre todos los ejecutores. Se crean una vez el controlador y luego los ejecutores acceden a ellas. Se transmitir conjuntos de datos completos en un cluster de spark para que los ejecutores tengan acceso.

In [16]:
rdd = sc.parallelize([item for item in range(10)])

In [17]:
uno = 1

In [18]:
# Para crear una variable broadcast
br_uno = sc.broadcast(uno)

In [20]:
# Para utilizar la variable
rdd1 = rdd.map(lambda x: x + br_uno.value)
rdd1.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

Las variables broadcast ocupan memoria en todos los ejecutores. Tenerlo en cuenta.

In [22]:
# Para eliminar variable broadcast de todos los ejecutores.
br_uno.unpersist()
# Si la variable se vuelve a utilizar, se retransmite a los ejecutores.
rdd1 = rdd.map(lambda x: x + br_uno.value) # Por lo anterior, se ejecuta sin problema

In [ ]:
# Para eliminar por completo del controlador y los ejecutores, es decir, queda inaccesible
br_uno.destroy()

### Acumuladores

Variables compartidas entre los ejecutores que normalmente se usan para agregar contadores al programa en Spark

In [27]:
# Para definir un acumulador
# 0 sería el valor inicial
acumulador = sc.accumulator(0)

In [28]:
rdd = sc.parallelize([2,4,6,8,10])

In [29]:
# Para agregar valor al acumulador acumulador.add()
# En este caso se usa el acumulador para calcular la suma de los valores del rdd
rdd.foreach(lambda x: acumulador.add(x))

In [30]:
# Para obtener el valor del acumulador
acumulador.value

30